### CS/ECE/ISyE 524 &mdash; Introduction to Optimization &mdash; Spring 2018 ###

# Project title goes here #

#### Student 1 (email address), Student 2 (email address), Student 3 (email address), and Student 4 (email address)

*****

### Table of Contents

1. [Introduction](#1.-Introduction)
1. [Mathematical Model](#2.-Mathematical-model)
1. [Solution](#3.-Solution)
1. [Results and Discussion](#4.-Results-and-discussion)
  1. [Optional Subsection](#4.A.-Feel-free-to-add-subsections)
1. [Conclusion](#5.-Conclusion)

## 1. Introduction ##

Here in America, commentators have long noted the structural factors creating permanent communities of underprivileged citizens. In recent years, we’ve seen a particular focus on lack of access to critical health care and prevention resources, with legislative actions and nonprofit initiatives ranging from the Affordable Care Act to Michelle Obama’s Let’s Move! campaign to reduce childhood obesity. One emerging area of investigation revolves around communities’ access to a variety high-quality, healthy foods. In our project, we aim to quantify these access issues by consider the optimal diets that can be constructed from the selection available at a variety of local different stores. 

In constructing our project, we realized that access to healthy nutritional foods depends on a variety of issues. How far away is the grocery store from your house? Do you even have a way of getting to the nearest grocery store? How can you construct a varied diet from the food available to you? Is it better to shop at more than one store given the variation in cost between stores? We sought to answer these questions by improving on a well-known study from economic literature: the Stigler diet.

George Stigler was an economist at Columbia University and the University of Chicago who revolutionized his field by quantitatively analyzing the impact of government economic regulations (previous economists’ analysis had focused on qualitative effects and theory). [1] In 1945, Stigler gathered data on a basic list of foods, nutrients, and prices, then played with the numbers to quantify the cheapest possible diet on the market. His work would inspire George Dantzig to develop the Simplex Algorithm just a few years later, one of the most efficient algorithms available today for solving general problems with linear constraint and objective functions. [2]

Stigler’s diet provided a very good baseline for understanding the price associated with a barebones minimalist diet. However, he didn’t take into account that food has to be bought in unit quantities (he considered only how many nutrients were contained in one dollar’s worth of flour, for instance) and he didn’t consider several additional constraints, like travel time and expense, differing food price/availability by store, and age and gender concerns. In our project, we sought to address both of these issues to provide a more complete picture of modern Americans’ access to good, nutritious food.

We got our nutrient data from the USDA’s Food Composition databases, available online. [3] We hypothesized different prices for different stores that varied based on season; we also created some estimate figures for gas prices and the like. In this report, we explain the theory behind the model in the Mathematical Model section and give the code we used in the code section. We discuss the key results immediately afterwards and provide some conclusions and directions for future study in the final section. We hope that this project will provide you the reader with a quantitative understanding of the tradeoffs between the best prices available and the time needed to travel to the stores that carry those best prices.

[1] http://www.econlib.org/library/Enc/bios/Stigler.html 
[2] https://research.googleblog.com/2014/09/sudoku-linear-optimization-and-ten-cent.html and https://web.archive.org/web/20160411141356/https://dl.dropboxusercontent.com/u/5317066/1990-dantzig-dietproblem.pdf 
[3] https://ndb.nal.usda.gov/ndb/search/list

## 2. Mathematical model ##

### The Simple Model ###

There will be only one source of food and only one age group (adult). Also, it is assumed that there is no additional cost in regards to transportation and that the climate remains constant.

Parameter definitions:

F = number of foods  
N = number of nutrients  
T = number of food categories  
f[i,j] = amount of nutrient j in food i  
n[i] = minimum daily requirement for nutrient i  
m[i] = maximum daily requirement for nutrient i  
o[i] = maximum food limit for food i

**Variable definitions:** 
x[i] = number of units of food i purchased  

**Constraints:**  
The amount of any given nutrient is greater than or equal to its lower limit and less than or equal to its upper limit.  
$ \sum_{i=1}^{F} x[i]*f[i,j] \leq m[j] \forall j \in N$  
$ \sum_{i=1}^{F} x[i]*f[i,j] \geq n[j] \forall j \in N$  

The amount of food should not exceed its upper limit.
$ \sum_{i=1}^{F} x[i]*f[i,j] \geq n[j] \forall j \in N$  

The amount from each food category is less than or equal to its upper limit and at least one.  
$\sum_{i=1}^{F} x[i] \leq p[i] $  

**Objective**
@objective(m, Min, sum[food[i] * food_prices[i] for i in 1:num_foods])



### The Full Model ###

This time, there are multiple stores as well as multiple age groups. We incorporate travel time to the stores as a tradeoff and consider the effects of seasonal variation in food pricing.  

**New parameters (in addition to those for simple model):**

A = number of age groups 

S = number of stores

Sd[i,j] = distance from location i to location j  

C = cost of gas per mile

c[i,j] = cost of food i at store j 

o[i,j] = order of stores travelleds (binary matrix)

**Constraints**  
This section is a bit different from the simple model section. Here we will solve several mixed integer programs and use the results to construct a Pareto curve considering the tradeoff between travel time and food pricing. We will also investigate vegetarian diets and age-appropriate diets.

## 3. Solution ##

### Simple Model (Scenario 1)

In [ ]:
using JuMP, Cbc

raw = readcsv("food_data.csv")
(m,n) = size(raw)

###Data
num_nutrients = 2:n
num_food = 17:m
food_categories = [:vegetables, :fruits, :grains, :meats, :dairy]


nutrients = raw[1,num_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[num_foods,1][:]             # the list of foods (convert to 1-D array)
min_req = Dict(zip(nutrients,raw[10,num_nutrients]))
max_req = Dict(zip(nutrients,raw[11,num_nutrients]))
max_quant = #not added to spreadsheet yet
food_nutrients = NamedArray(raw[num_food,num_nutrient], (foods,nutrients), ("foods","nutrients"));
food_prices = #not added to spreadsheet yet

m = Model(solver=CbcSolver())

##Decision varaible
@variable(m, food[1:num_food])

##Constraints

#Min/Max nutrients
@constraint(m, min_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) >= min_req[j])
@constraint(m, max_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) <= max_req[j])

#Max food amount
@constraint(m, max_food[i in 1:num_foods], sum(food[i] * max_quant[i]))

#Sum of each food category must be at least 1
#TODO

##Objective
@objective(m, Min, sum[food[i] * food_prices[i] for i in 1:num_foods])
solve(m)

### Full Model

In [ ]:
using JuMP, Cbc

raw = readcsv("food_data.csv")
(m,n) = size(raw)

###Data
num_nutrients = 2:n
num_food = 17:m
food_categories = [:vegetables, :fruits, :grains, :meats, :dairy]
age_groups = [:child, :teenager, :adult]
locations = #add locations
gas = 3

nutrients = raw[1,num_nutrients][:]   # the list of nutrients (convert to 1-D array)
foods = raw[num_foods,1][:]             # the list of foods (convert to 1-D array)
min_req = Dict(zip(nutrients,raw[10,num_nutrients]))
max_req = Dict(zip(nutrients,raw[11,num_nutrients]))
max_quant = #not added to spreadsheet yet
food_nutrients = NamedArray(raw[num_food,num_nutrient], (foods,nutrients), ("foods","nutrients"));
food_prices = #not added to spreadsheet yet
distances = #no data yet


m = Model(solver=CbcSolver())

##Decision varaible
@variable(m, food[1:num_food])
@variable(m, path[i in locations, j in locations])

##Constraints

#Min/Max nutrients
@constraint(m, min_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) >= min_req[j])
@constraint(m, max_nutrient[j in 1:num_nutrients], sum(food[i] * food_nutrients[i,j] for i in 1:num_foods) <= max_req[j])

#Max food amount
@constraint(m, max_food[i in 1:num_foods], sum(food[i] * max_quant[i]))

#Sum of each food category must be at least 1
#TODO

##Objective
@objective(m, Min, sum[food[i] * food_prices[i] for i in 1:num_foods] + sum(path[k,l] * distances[k,l] * gas for k in 1 in locations, l in locations))
solve(m)

Remember to make sure your code compiles! I will be running your code!

## 4. Results and discussion ##

Here, you display and discuss the results. Show figures, plots, images, trade-off curves, or whatever else you can think of to best illustrate your results. The discussion should explain what the results mean, and how to interpret them. You should also explain the limitations of your approach/model and how sensitive your results are to the assumptions you made.

Use plots (see `PyPlot` examples from class), or you can display results in a table like this:

| Tables        | Are           | Cool  |
| ------------- |:-------------:| -----:|
| col 3 is      | right-aligned |\$1600 |
| col 2 is      | centered      |  \$12 |
| zebra stripes | are neat      |   \$1 |

### 4.A. Feel free to add subsections

#### 4.A.a. or subsubsections

## 5. Conclusion ##

Summarize your findings and your results, and talk about at least one possible future direction; something that might be interesting to pursue as a follow-up to your project.

### TODO:
   - Price scaling factor
   - Add taste factor?